# Combine results from 11v11 (Football League) and Soccerbase (Non-League)

In [13]:
import pandas as pd

# Import results from 11v11
df_1 = pd.read_csv("../data/11v11_results.csv", parse_dates=["game_date"])
df_1 = df_1[~df_1.season.isin(["2015/16", "2016/17", "2017/18"])]

# Import National League results from Soccerbase
df_2 = pd.read_csv("../data/soccerbase_nat_league_results.csv", parse_dates=["game_date"])

# Merge both datasets
df = pd.concat([df_1, df_2]).sort_values("game_date", ascending=False).reset_index(drop=True)

### Add `league_tier`

In [14]:
league_tier_map = {
        'National League': 5,
        'Football Conference Play-off': 5,
        'League Two': 4,
        'League Two Play-Offs': 4,
        'League Division Four': 4,
        'League One': 3,
        'League One Play-Offs': 3,
        'League Division 2': 3,
        'League Division Three': 3,
        'League Division Three Play-Offs': 3,
        'Division Three (North)': 3,
        'League Division 1': 2,
        'League Division 1 Play-Offs': 2,
        'League Division Two': 2,
    }

df['league_tier'] = df.competition.map(league_tier_map)

### Add `generic_comps_map`

In [15]:
generic_comps_map = {
    "Anglo-Italian Cup": "Anglo-Italian Cup",
    'Associate Members Cup': "Associate Members' Cup",
    'Freight Rover Trophy': "Associate Members' Cup", 
    'Sherpa Vans Trophy': "Associate Members' Cup", 
    'Leyland Daf Cup': "Associate Members' Cup", 
    'LDV Vans Trophy': "Associate Members' Cup", 
    'Johnstones Paint Trophy': "Associate Members' Cup",
    'Football League Trophy': "Associate Members' Cup",
    "FA Cup": "FA Cup",
    "FA Trophy": "FA Trophy",
    "Zenith Data Systems Cup": "Full Members' Cup",
    "Division Three (North)": "Football League",
    "League Division 1": "Football League",
    "League Division 1 Play-Offs": "Football League",
    "League Division 2": "Football League",
    "League Division Four": "Football League",
    "League Division Three": "Football League",
    "League Division Three Play-Offs": "Football League",
    "League Division Two": "Football League",
    "League One": "Football League",
    "League One Play-Offs": "Football League",
    "League Two": "Football League",
    "League Two Play-Offs": "Football League",
    "League Cup": "League Cup",
    "Football Conference Play-off": "Non-League",
    "National League": "Non-League",
    "North Region War League": "War League",
    "North Region War League (Second Championship)": "War League",
    "West Region War League": "War League"
}

df['generic_comp'] = df.competition.map(generic_comps_map)

### Add `game_type`

In [16]:
df.loc[(df.generic_comp.isin(["Football League", "Non-League"])), "game_type"] = "League"
df.loc[((df.league_tier.isna()) & (~df.competition.str.contains("War League"))), "game_type"] = "Cup"
df.loc[(df.competition.str.contains("Play-")), "game_type"] = "League Play-Off"
df.loc[(df.competition.str.contains("War League")), 'game_type'] = "War League"

### Add `goal_diff`

In [17]:
df['goal_diff'] = df.goals_for.astype("int") - df.goals_against.astype("int")

### Add `ssn_game_no` and `ssn_comp_game_no`

In [18]:
df['ssn_game_no'] = df.sort_values(by=['game_date']).groupby(['season']).cumcount() + 1
df['ssn_comp_game_no'] = df.sort_values(by=['game_date']).groupby(['season', 'competition']).cumcount() + 1

### Add `weekday`

In [19]:
df['weekday'] = pd.to_datetime(df.game_date).dt.day_name()

### Add `manager`

In [20]:
def find_manager_on_date(input_date):
    input_date = pd.Timestamp(input_date)
    try:
        manager_index = managers_df.apply(lambda x : (input_date >= x.manager_start_date) & (input_date <= x.manager_end_date), axis = 1)
        manager = managers_df[manager_index]['manager_name'].iloc[0]
    except:
        manager = 'Unknown'
    return manager

# managers_df = pd.read_csv('../../scrape-managers/data/managers_df.csv')
# managers_df.manager_start_date = pd.to_datetime(managers_df.manager_start_date)
# managers_df.manager_end_date = pd.to_datetime(managers_df.manager_end_date)

managers_df = pd.read_html("https://www.soccerbase.com/teams/team.sd?team_id=2598&teamTabs=managers")[1].rename(columns = {"Unnamed: 0": "manager_name", "FROM": "manager_start_date", "TO": "manager_end_date"})
managers_df.manager_start_date = pd.to_datetime(managers_df.manager_start_date)
managers_df.manager_end_date = managers_df.apply(lambda x: pd.to_datetime("today") if x.manager_end_date == "Present" else pd.to_datetime(x.manager_end_date), axis=1)

df['manager'] = df.apply(lambda row : find_manager_on_date(row.game_date), axis = 1)

### Change venue of all Wembley games to N(eutral)

In [21]:
# Update venue of Wembley finals to "N" (Neutral)
df.loc[df.stadium == "Wembley Stadium", "venue"] = "N"

# Make fixes to dataframe

### Update `df` with attendances from `manual_attendances.csv`

In [22]:
# Read in manually collected attendances
new_attendances = pd.read_csv("../data/manual_attendances.csv", parse_dates=['game_date'])
new_attendances = new_attendances[~new_attendances.attendance.isna()]

# Filter df for games that will be updated from new_attendances df
# This is necessary to make the length of both dataframes match
updates = df[df.game_date.isin(new_attendances.game_date)].copy()

# Insert the attendance for rows where the game_date is in both dataframes
updates.attendance = new_attendances[new_attendances.game_date.isin(updates.game_date)].attendance.values

# Update the original df with our freshly updated updates df
df.update(updates)

### Update `df` with manager fixes from `manager_updates.csv`

In [23]:
# Read in manual manager updates
new_managers = pd.read_csv("../data/manager_updates.csv", parse_dates=['game_date'])

# Filter df for games that will be updated from new_managers df
# This is necessary to make the length of both dataframes match
man_updates = df[df.game_date.isin(new_managers.game_date)].copy()

# Insert the manager naems for rows where the game_date is in both dataframes
man_updates.manager = new_managers[new_managers.game_date.isin(man_updates.game_date)].manager.values

# Update the original df with our freshly updated man_updates df
df.update(man_updates)

### Output fixed dataframe to `results_df.csv`

In [12]:
df.to_csv("../output/results_df.csv", index=False)